In [1]:
import os

from pyspark.sql import SparkSession, functions as F
import logging

from utils import read_geo, read_events, add_closest_city, add_distance

In [13]:
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3"
os.environ["YARN_CONF_DIR"] = "/etc/hadoop/conf"
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
os.environ["HADOOP_CONF_DIR"] = "/etc/hadoop/conf"

In [2]:
# DEV Constants
GEO_DIR = "/user/solovyovyu/geo.csv"
EVENTS_DIR = "/user/solovyovyu/data/geo/events"
OUT_PATH = "/user/solovyovyu/analytics"

# PROD Constants
# EVENTS_DIR = "/user/master/data/geo/events"

In [3]:
def setup_logging():
    """Configure logging"""
    logger = logging.getLogger(__name__)
    logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
    return logger

logger = setup_logging()

In [4]:
try:
    spark = SparkSession.builder \
        .master("local") \
        .appName("Mart User") \
        .getOrCreate()
    logger.info("SparkSession создан успешно.")
except Exception as e:
    logger.error(f"Ошибка при создании SparkSession: {e}")
    raise

your 131072x1 screen size is bogus. expect trouble
25/02/12 17:09:02 WARN Utils: Your hostname, DESKTOP-BBBONO9 resolves to a loopback address: 127.0.1.1; using 172.20.57.228 instead (on interface eth0)
25/02/12 17:09:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/12 17:09:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2025-02-12 17:09:07,599 - INFO - SparkSession создан успешно.


In [5]:
geo_df = read_geo(GEO_DIR, spark, logger)

2025-02-12 17:09:15,036 - INFO - Geo data is read from /user/solovyovyu/geo.csv.


In [6]:
messages_df = read_events("message", EVENTS_DIR, spark, logger)

2025-02-12 17:09:16,079 - INFO - Events message are read from /user/solovyovyu/data/geo/events.


In [7]:
reaction_df = read_events("reaction", EVENTS_DIR, spark, logger)

2025-02-12 17:09:16,698 - INFO - Events reaction are read from /user/solovyovyu/data/geo/events.


In [8]:
subscription_df = read_events("subscription", EVENTS_DIR, spark, logger)

2025-02-12 17:09:17,140 - INFO - Events subscription are read from /user/solovyovyu/data/geo/events.


In [9]:
message_with_distance = add_distance(messages_df, geo_df, logger)

In [10]:
messages_with_city = add_closest_city(message_with_distance, logger)

messages_with_city.cache()

messages_with_city.show()

+----------+-------+--------------------+-----------+-------------------+
|message_id|user_id|            datetime|       city|           timezone|
+----------+-------+--------------------+-----------+-------------------+
|      1519|  78470|2021-05-04 00:22:...|    Bendigo|Australia/Melbourne|
|      7802| 117522|2021-05-05 21:19:...|   Brisbane| Australia/Brisbane|
|     27461|   5193|2021-05-05 11:27:...|   Adelaide| Australia/Adelaide|
|     66514|  99491|2021-05-04 21:49:...|  Melbourne|Australia/Melbourne|
|     86571|  77248|2021-05-05 11:28:...|    Bendigo|Australia/Melbourne|
|    144030| 118361|2021-05-05 20:38:...|     Darwin|   Australia/Darwin|
|    213311| 139927|2021-05-04 13:59:...|      Perth|    Australia/Perth|
|    224842|  95820|2021-05-05 17:28:...|   Brisbane| Australia/Brisbane|
|    234156| 120215|2021-05-05 12:26:...| Cranbourne|Australia/Melbourne|
|    236513|  46233|2021-05-04 19:11:...|     Cairns| Australia/Brisbane|
|    257217|  32179|2021-05-04 23:53:.

In [11]:
messages_with_weeks = messages_with_city \
    .withColumn("event_date", F.to_date(F.col("datetime"))) \
    .withColumn("year", F.year("event_date")) \
    .withColumn("month", F.month("event_date")) \
    .withColumn("week", F.weekofyear("event_date"))

In [22]:
messges_by_weeks = messages_with_weeks.groupBy("year", "week", "city") \
    .agg(
        F.count("message_id").alias("week_message")
    )

In [23]:
messges_by_month = messages_with_weeks \
    .groupBy("year", "month", "city") \
    .agg(
        F.count("message_id").alias("month_message")
    )

In [24]:
subscription_with_city = add_closest_city(subscription_df, geo_df, logger)

subscription_with_city.show()

TypeError: add_closest_city() takes 2 positional arguments but 3 were given